<a href="https://colab.research.google.com/github/harmeet-singh-bagga/ExplainableNLPsuite/blob/main/OpenGV_InternVL2_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install decord

In [2]:
!pip install einops flash_attn timm

In [3]:
!pip install -U bitsandbytes

In [4]:
!pip install transformers==4.37.2

In [5]:
import math
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

In [6]:
def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

In [7]:
def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

In [8]:
def dynamic_preprocess(image, min_num=1, max_num=6, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

In [9]:
def load_image(image_file, input_size=448, max_num=6):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [10]:
def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    num_layers = {'InternVL2-8B': 32, 'InternVL2-26B': 48,
                  'InternVL2-40B': 60, 'InternVL2-Llama3-76B': 80}[model_name]
    # Since the first GPU will be used for ViT, treat it as half a GPU.
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

In [11]:
# OpenGVLab/InternVL2-Llama3-76B
path = 'OpenGVLab/InternVL2-8B'
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True).eval().cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2-8B:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 37.06 MiB is free. Process 19193 has 14.71 GiB memory in use. Of the allocated memory 14.45 GiB is allocated by PyTorch, and 168.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
# set the max number of tiles in `max_num`
pixel_values = load_image('test_l1.png', max_num=6).to(torch.bfloat16).cuda()

generation_config = dict(
    num_beams=1,
    max_new_tokens=1024,
    do_sample=False,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
question = '<image>\nPlease describe the inference from the image in detail.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}')
print(f'Assistant: {response}')

User: <image>
Please describe the inference from the image in detail.
Assistant: The image is a map showing the 30-day total rainfall in millimeters (mm) for a specific period from June 2020 to April 2024. The map uses a color gradient to represent different levels of rainfall across the region. Here is a detailed description of the inference from the image:

1. **Color Gradient and Rainfall Levels**:
   - **Dark Blue**: Represents the highest rainfall levels, with values ranging from 100 to 150 mm.
   - **Medium Blue**: Indicates moderate rainfall, with values from 50 to 100 mm.
   - **Light Blue**: Represents lower rainfall levels, with values from 0 to 50 mm.
   - **Green**: Indicates areas with no rainfall or very low rainfall, with values from 0 to 25 mm.
   - **Yellow**: Represents areas with very low rainfall, with values from 0 to 25 mm.

2. **Geographical Distribution**:
   - **Northern and Eastern Regions**: These areas show a mix of green and light blue, indicating lower to 

In [ ]:
question = '<image>\nCan you identify from the map which region of the world this is?'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}')
print(f'Assistant: {response}')

User: <image>
Can you identify from the map which region of the world this is?
Assistant: The map shows a region in the Southern Hemisphere, with the South American continent visible. The map is focused on the western part of South America, including countries such as Chile, Argentina, and parts of Peru and Bolivia. The map is color-coded to represent rainfall data, with different colors indicating different levels of rainfall.


In [ ]:
pixel_values = load_image('/content/test1.png', max_num=6).to(torch.bfloat16).cuda()

generation_config = dict(
    num_beams=1,
    max_new_tokens=1024,
    do_sample=False,
)

In [ ]:
question = '<image>\nPlease describe the inference from the image in detail. This image is about the runs scored by Indian Batsmen in the 2011 ICC ODI Cricket World cup.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}')
print(f'Assistant: {response}')

User: <image>
Please describe the inference from the image in detail. This image is about the runs scored by Indian Batsmen in the 2011 ICC ODI Cricket World cup.
Assistant: The image is a bar chart titled "Runs scored by Indian Batsmen in front of Sri-Lankan Bowlers" from the 2011 ICC ODI Cricket World Cup. The chart is divided into three sections, each representing a different bowler: Gambhir, Dhoni, and Tendulkar. Each section contains bars representing the runs scored by Indian batsmen against the respective bowler, categorized by the bowler's name and the batsman's name.

### Gambhir Section:
- **TM Dilshan**: The bar for TM Dilshan is the shortest, indicating the least runs scored.
- **SL Malinga**: The bar for SL Malinga is slightly longer than TM Dilshan's, showing a small increase in runs.
- **S Randiv**: The bar for S Randiv is longer than the previous two, indicating more runs scored.
- **NLTC Perera**: The bar for NLTC Perera is the longest in this section, showing the high

In [ ]:
pixel_values = load_image('/content/example.png', max_num=6).to(torch.bfloat16).cuda()

generation_config = dict(
    num_beams=1,
    max_new_tokens=1024,
    do_sample=False,
)

question = '<image>\nPlease describe the inference from the image in detail. Give me a detailed, professional response.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}')
print(f'Assistant: {response}')

User: <image>
Please describe the inference from the image in detail. Give me a detailed, professional response.
Assistant: The image is a bar chart titled "Compare sales strategy." The chart compares the sales performance of five different products (A, B, C, D, and E) across eight different strategies (1 through 8). Each product is represented by a different color: Product A is blue, Product B is orange, Product C is yellow, Product D is green, and Product E is red. The vertical axis represents the sales value, ranging from 0 to 900, with increments of 100.

### Analysis of the Chart:

1. **Product A (Blue)**:
   - Strategy 1: Approximately 200
   - Strategy 2: Approximately 300
   - Strategy 3: Approximately 400
   - Strategy 4: Approximately 500
   - Strategy 5: Approximately 600
   - Strategy 6: Approximately 700
   - Strategy 7: Approximately 800
   - Strategy 8: Approximately 900

2. **Product B (Orange)**:
   - Strategy 1: Approximately 100
   - Strategy 2: Approximately 200
   

In [ ]:
question = '<image>\nPlease tell me more about Product E'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}')
print(f'Assistant: {response}')

User: <image>
Please tell me more about Product E
Assistant: Product E is represented by the dark gray color in the bar chart. The chart compares the sales strategies of different products (A, B, C, D, E, F, and G) across eight strategies. Product E consistently shows a sales volume that is higher than Product A but lower than Products B, C, D, and F. 

Here is a detailed breakdown of Product E's performance across the strategies:

1. **Strategy 1**: Product E has a sales volume slightly above 300.
2. **Strategy 2**: Product E's sales volume is around 400.
3. **Strategy 3**: Product E's sales volume is slightly above 400.
4. **Strategy 4**: Product E's sales volume is around 500.
5. **Strategy 5**: Product E's sales volume is slightly above 500.
6. **Strategy 6**: Product E's sales volume is around 600.
7. **Strategy 7**: Product E's sales volume is slightly above 600.
8. **Strategy 8**: Product E's sales volume is around 700.

Product E's sales volume increases steadily across the str

In [ ]:
# # multi-image multi-round conversation, combined images (多图多轮对话，拼接图像)
# pixel_values1 = load_image('./examples/image1.jpg', max_num=6).to(torch.bfloat16).cuda()
# pixel_values2 = load_image('./examples/image2.jpg', max_num=6).to(torch.bfloat16).cuda()
# pixel_values = torch.cat((pixel_values1, pixel_values2), dim=0)

# question = '<image>\nDescribe the two images in detail.'
# response, history = model.chat(tokenizer, pixel_values, question, generation_config,
#                                history=None, return_history=True)

# question = 'What are the similarities and differences between these two images.'
# response, history = model.chat(tokenizer, pixel_values, question, generation_config,
#                                history=history, return_history=True)
# print(f'User: {question}')
# print(f'Assistant: {response}')

# # multi-image multi-round conversation, separate images (多图多轮对话，独立图像)
# pixel_values1 = load_image('./examples/image1.jpg', max_num=6).to(torch.bfloat16).cuda()
# pixel_values2 = load_image('./examples/image2.jpg', max_num=6).to(torch.bfloat16).cuda()
# pixel_values = torch.cat((pixel_values1, pixel_values2), dim=0)
# num_patches_list = [pixel_values1.size(0), pixel_values2.size(0)]

# question = 'Image-1: <image>\nImage-2: <image>\nDescribe the two images in detail.'
# response, history = model.chat(tokenizer, pixel_values, question, generation_config,
#                                num_patches_list=num_patches_list,
#                                history=None, return_history=True)
# print(f'User: {question}')
# print(f'Assistant: {response}')

# question = 'What are the similarities and differences between these two images.'
# response, history = model.chat(tokenizer, pixel_values, question, generation_config,
#                                num_patches_list=num_patches_list,
#                                history=history, return_history=True)
# print(f'User: {question}')
# print(f'Assistant: {response}')

# # batch inference, single image per sample (单图批处理)
# pixel_values1 = load_image('./examples/image1.jpg', max_num=6).to(torch.bfloat16).cuda()
# pixel_values2 = load_image('./examples/image2.jpg', max_num=6).to(torch.bfloat16).cuda()
# num_patches_list = [pixel_values1.size(0), pixel_values2.size(0)]
# pixel_values = torch.cat((pixel_values1, pixel_values2), dim=0)

# questions = ['<image>\nDescribe the image in detail.'] * len(num_patches_list)
# responses = model.batch_chat(tokenizer, pixel_values,
#                              num_patches_list=num_patches_list,
#                              questions=questions,
#                              generation_config=generation_config)
# for question, response in zip(questions, responses):
#     print(f'User: {question}')
#     print(f'Assistant: {response}')

# # video multi-round conversation (视频多轮对话)
# def get_index(bound, fps, max_frame, first_idx=0, num_segments=32):
#     if bound:
#         start, end = bound[0], bound[1]
#     else:
#         start, end = -100000, 100000
#     start_idx = max(first_idx, round(start * fps))
#     end_idx = min(round(end * fps), max_frame)
#     seg_size = float(end_idx - start_idx) / num_segments
#     frame_indices = np.array([
#         int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
#         for idx in range(num_segments)
#     ])
#     return frame_indices

# def load_video(video_path, bound=None, input_size=448, max_num=1, num_segments=32):
#     vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
#     max_frame = len(vr) - 1
#     fps = float(vr.get_avg_fps())

#     pixel_values_list, num_patches_list = [], []
#     transform = build_transform(input_size=input_size)
#     frame_indices = get_index(bound, fps, max_frame, first_idx=0, num_segments=num_segments)
#     for frame_index in frame_indices:
#         img = Image.fromarray(vr[frame_index].asnumpy()).convert('RGB')
#         img = dynamic_preprocess(img, image_size=input_size, use_thumbnail=True, max_num=max_num)
#         pixel_values = [transform(tile) for tile in img]
#         pixel_values = torch.stack(pixel_values)
#         num_patches_list.append(pixel_values.shape[0])
#         pixel_values_list.append(pixel_values)
#     pixel_values = torch.cat(pixel_values_list)
#     return pixel_values, num_patches_list


# video_path = './examples/red-panda.mp4'
# # pixel_values, num_patches_list = load_video(video_path, num_segments=32, max_num=1)
# pixel_values, num_patches_list = load_video(video_path, num_segments=8, max_num=1)
# pixel_values = pixel_values.to(torch.bfloat16).cuda()
# video_prefix = ''.join([f'Frame{i+1}: <image>\n' for i in range(len(num_patches_list))])
# question = video_prefix + 'What is the red panda doing?'
# # Frame1: <image>\nFrame2: <image>\n...\nFrame31: <image>\n{question}
# response, history = model.chat(tokenizer, pixel_values, question, generation_config,
#                                num_patches_list=num_patches_list,
#                                history=None, return_history=True)
# print(f'User: {question}')
# print(f'Assistant: {response}')

# question = 'Describe this video in detail. Don\'t repeat.'
# response, history = model.chat(tokenizer, pixel_values, question, generation_config,
#                                num_patches_list=num_patches_list,
#                                history=history, return_history=True)
# print(f'User: {question}')
# print(f'Assistant: {response}')

## Microsoft Florence

In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The repository for microsoft/Florence-2-large contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Florence-2-large.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
def run_example(task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to('cuda', torch.float16)
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer

In [ ]:
import requests
from PIL import Image
url = "/content/IMG_0064.JPG"
image = Image.open(url).convert("RGB")

In [ ]:
image

In [ ]:
task_prompt = '<MORE_DETAILED_CAPTION>'
run_example(task_prompt)

{'<MORE_DETAILED_CAPTION>': 'The image is a line graph that shows the trend of average runs per match for the United States from 1972 to 2024. The x-axis represents the years, starting from 1972 and ending in 2024, with the y-axis representing the average run per match.\n\nThere are six lines in the graph, each representing a different year. The first line is blue, the second line is black, the third line is gray, and the fourth line is white. The lines are arranged in a vertical axis, with each line representing a year. \n\nThe line on the left side of the graph has a vertical line, while the line in the middle has a horizontal line. The line at the top of the line is slightly curved, indicating that the trend has been steadily increasing over time. The graph also has a legend at the bottom that explains the meaning of each line.'}

In [ ]:
prompt = "<OD>"
run_example(prompt)

{'<OD>': {'bboxes': [[3396.031982421875,
    1632.780029296875,
    3624.639892578125,
    1744.4520263671875],
   [2956.864013671875,
    1561.716064453125,
    3161.407958984375,
    1673.3880615234375],
   [2632.0, 1548.1800537109375, 2806.4638671875, 1639.5479736328125],
   [2974.911865234375, 377.3160095214844, 3636.671875, 1741.0679931640625],
   [2613.951904296875,
    455.14801025390625,
    3161.407958984375,
    1666.6199951171875]],
  'labels': ['footwear', 'footwear', 'footwear', 'person', 'person']}}